앞장에서 가중치 매개변수의 기울기는 수치미분을 사용했지만, 이는 시간이 오래걸린다는게 단점!    

이번장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 '오차역전파법'을 배운다.

# 5. 오차역전파법

# 5.1.1 계산 그래프로 풀다

문제1 : 슈퍼에서 1개 100원인 사과를 2개 사고 소비세가 10%일 때, 지불 금액은?
  
100 * 2 * 1.1 = 220
       200
100 → × →  × → 220
2   ↗     ↗
       1.1
 
문제2 : 슈퍼에서 1개 100원인 사과를 2개 사고 150원인 귤을 3개 샀다. 소비세가 10%일 때, 지불 금액은?
((100 * 2) + (150 * 3)) * 1.1 = 715
3   ↘
150 → × ↘
100 → × →  + → × → 715
2   ↗          ↗
           1.1
순전파forward propagation : 계산을 왼쪽에서 오른쪽으로 진행
역전파backward propagation : 계산을 반대 방향으로 진행. 미분을 효율적으로 계산할 수 있다.
"""

## 곱셈계층

모든 계층은 forward와 backward라는 공통의 메서드(인터페이스)를 갖도록 구현할 것이다.  
forward : 순전파
backward : 역전파

In [3]:
class MulLayer: # 곱셈 노드
    def __init__(self):   # x와 y를 초기화 한다.
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):    # dout : 미분. 순전파때의 값을 서로 바꿔 곱한 후 하류로 흘린다.
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

### 문제1 순전파

In [6]:
 # 문제1의 예시
apple = 100
apple_num = 2
tax = 1.1

    # 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

    # 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  # mullayger라는 곱셈 노드에서 forward라는 메서드를 갖음
price = mul_tax_layer.forward(apple_price, tax)

print(price)  # 220.0

220.00000000000003


### 문제1 역전파

In [9]:
    # 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)  # 2.2 110.0 200

2.2 110.00000000000001 200


## 덧셈계층

In [11]:
# 5.4.2 덧셈 계층
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy


문제 2 순전파

In [12]:
orange = 150
orange_num = 3

    # 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [14]:
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax) 
print(price)  # 715.0

715.0000000000001


In [15]:
# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dornage, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_num, dapple, dornage, dorange_num, dtax)
    # 110.0 2.2 3.3 165.0 650

110.00000000000001 2.2 3.3000000000000003 165.0 650


## Relu 함수와 시그모이드 함수

In [16]:
import numpy as np

# 5.5.1 ReLU 계층

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [17]:
# 5.5.2 Sigmoid 계층

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


## 5.6 Affine/Softmax 계층 구현하기

### Affine
신경망의 순전파 때 수행하는 행령의 곱은 기하학에서는 '어파인 변환'이라고 하는데, 어파인 변환을 수행하는 처리를 '어파인 계층'이라는 이름으로 구현한다.

"""
Affine 계층의 계산 그래프
X, W, B는 행렬(다차원 배열)

1. ∂L/∂X = ∂L/∂Y·W^T
   (2,)    (3,)  (3,2)
2. ∂L/∂W = X^T·∂L/∂Y
   (2,3)  (2,1)(1,3)
3. ∂L/∂B = ∂L/∂Y
   (3,)    (3,)
W^T : W의 전치행렬(W가 (2,3)이라면 W^T는(3,2)가 된다.
X = (x0, x1, x2, ..., xn)
∂L/∂X = (∂L/∂x0, ∂L/∂x1, ∂L/∂x2, ..., ∂L/∂xn)   
따라서 X와 ∂L/∂X의 형상은 같다.
"""


"""
입력 데이터로 X 하나만이 아니라 데이터 N개를 묶어 순전파하는 배치용 계층을 생각
배치용 Affine 계층의 계산 그래프
X의 형상이 (N,2)가 됨.
1. ∂L/∂X = ∂L/∂Y·W^T  
   (N,2)   (N,3) (3,2)  
2. ∂L/∂W = X^T·∂L/∂Y  
   (2,3)  (2,N)(N,3)  
3. ∂L/∂B = ∂L/∂Y의 첫 번째 축(0축, 열방향)의 합.  
   (3,)    (N,3)    
편향을 더할 때에 주의해야 한다. 순전파 때의 편향 덧셈은 X·W에 대한 편향이  
각 데이터에 더해진다. 예를 들어 N=2일 경우 편향은 두 데이터 각각에 더해진다.
"""

In [22]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### 5.6.3 Softmax-with-Loss 계층
소프트맥스 계층 : 입력 값을 정규화(출력의 합이 1이 되도록 변경)하여 출력 : 출력층에서 하용하는 함수!!
학습과 추론 중 학습에서 주로 사용  
소프트맥스 계층과 손실 함수(교차 엔트로피 오차)를 포함해 계산 그래프를 그림  


![](.\png.png)

cf. 신경망에서 수행하는 작업은 학습과 추론(inference) 두 가지이다. 추론할때는 일반적으로 softmax 계층을 사용하지 않는다.   
예컨데 신경망을 추론할때는 위 그래프에서 Affine 계층의 출력을 인식 결과로 이용을 많이한다.  
Affine 은 점수(score)라 하고,  
추론이면 : 답을 하나만 내는 경우에 가장 높은 점수만 알면 되니 Softmax  계층은 필요가 없고,  
학습이면 : Softmax  계층은 필요하다.

In [1]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None     # softmax의 출력
        self.t = None     # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)  # 3.5.2, 4.2.2에서 구현
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = self.y - self.t / batch_size

        return dx


cf. 엔트로피가 크다는것은 사건의 확률이 낮다는것. "어떤 상태에서의 불확실성"ㅡ  
오차 제곱(SE) 시리즈와의 차이점은, 오차제곱 시리즈는 회귀식처럼 값의 흩어진 정도를 출력한다면    
교차 엔트로피 오차는 정답 레이블에서 정답에 해당하는 위치의 확률의 로그값이 출력되게 된다.

## 5.7 오차역전파법 구현하기
신셩망의 구성요소를 모듈화하여 계층으로만 구현했기 때문에 구축이 쉬원진다!

In [1]:
import sys
import os
import numpy as np
from collections import OrderedDict
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient


In [6]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
        weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()    # 마지막 층은 소프트맥스위드로스!!

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
        # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads




신경망의 계층을 순서가 있는 딕셔너리에서 보관,
따라서 순전파때는 추가한 순서대로 각 계층의 forward()를 호출하기만 하면 된다.
역전파때는 계층을 반대 순서로 호출하기만 하면 된다.
신경망의 구성 요소를 모듈화하여 계층으로 구현했기 때문에 구축이 쉬워진다.

### 5.7.3 오차역전파법으로 구한 기울기 검증하기


기울기를 구하는데는 두 가지 방법이 있다.
1. 수치 미분 : 느리다. 구현이 쉽다.
2. 해석적으로 수식을 풀기(오차 역전파법) : 빠르지만 실수가 있을 수 있다.  
두 기울기 결과를 비교해서 오차역전파법을 제대로 구현했는지 검증한다.
이 작업을 기울기 확인gradient check라고 한다.
